In [9]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.externals import joblib

import random
from collections import defaultdict

import pandas as pd

Setting environment for Gordon


[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


In [3]:
train_patches_jpg_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_spm_trainPatches'

In [4]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
colors[labels_index['BackG']] = 1.

In [5]:
output_dir = '/oasis/projects/nsf/csd395/yuncong/Brain/learning/sift'
M = 200 # vocabulary size
colors = np.vstack([(0,0,0), np.random.randint(0, 255, (M, 3))])

In [7]:
def compute_vocabulary():
    '''
    Load vocabulary (as a sklearn.KMeans object)
    '''

    if os.path.exists(output_dir + '/vocab.pkl'):

        # Load vocabulary
        vocabulary = joblib.load(output_dir + '/vocab.pkl')

    else:

        if os.path.exists(output_dir + '/sift_descriptors_pool_arr.bp'):

            # Load descriptor pool
            descriptors_pool_arr = bp.unpack_ndarray_file(output_dir + '/sift_descriptors_pool_arr.bp')

            t = time.time()

            vocabulary = KMeans(init='random', n_clusters=M, n_init=10)
            vocabulary.fit(descriptors_pool_arr)

            sys.stderr.write('sift: %.2f seconds\n' % (time.time() - t)) # 300 seconds

            cluster_centers = vocabulary.cluster_centers_

            joblib.dump(vocabulary, output_dir + '/vocab.pkl')

        else:

            # Generate SIFT descriptor pool
            descriptors_pool = []

            sift = cv2.SIFT();

            dm = DataManager(stack='MD589')

            for sec in range(first_detect_sec, last_detect_sec+1, 10):

                print sec

                xmin, ymin, w, h = detect_bbox_lookup['MD589'] # in thumbnail resolution
                # convert to lossless resolution
                xmin = xmin * 32
                ymin = ymin * 32
                w = w * 32
                h = h * 32
                xmax = xmin + w - 1
                ymax = ymin + h - 1

                dm.set_slice(sec)
                dm._load_image(versions=['rgb-jpg'])
                img = dm.image_rgb_jpg[ymin:ymax+1, xmin:xmax+1]

                keypoints, descriptors = sift.detectAndCompute(img, None)

                n = 1000
                random_indices = np.random.choice(range(len(descriptors)), n, replace=False)

                descriptors_pool.append(descriptors[random_indices])

            descriptors_pool_arr = np.vstack(descriptors_pool)
            print len(descriptors_pool_arr), 'in descriptor pool'

            bp.pack_ndarray_file(descriptors_pool_arr, output_dir + '/sift_descriptors_pool_arr.bp')
            
    return vocabulary


def conmpute_labelmap(stack, sec):
    
    labelmap_fp = output_dir + '/%(stack)s/%(stack)s_%(sec)04d_labelmap.hdf' % {'stack': stack, 'sec': sec}
    
    if os.path.exists(labelmap_fp):
        
        # Load labelmap
        labelmap = load_hdf(labelmap_fp)
    else:
        
        sift = cv2.SIFT()
        
        # Compute keypoints and assign labels
        
        image_path = DataManager.get_image_filepath(stack=stack, section=sec, version='rgb-jpg')
        image = imread(image_path)
        
        xmin, ymin, w, h = detect_bbox_lookup[stack] # in thumbnail resolution
        # convert to lossless resolution
        xmin = xmin * 32
        ymin = ymin * 32
        w = w * 32
        h = h * 32
        xmax = xmin + w - 1
        ymax = ymin + h - 1
        
        img = image[ymin:ymax+1, xmin:xmax+1].copy()
        
        t = time.time()
        keypoints, descriptors = sift.detectAndCompute(img, None); # 128 dim descriptor ～ 120 seconds
        sys.stderr.write('sift: %.2f seconds\n' % (time.time() - t)) 

        keypoints_arr = np.array([k.pt for k in keypoints])
        print len(keypoints), 'keypoints' # ~ 500k

        t = time.time()
        vocabulary = compute_vocabulary()
        sys.stderr.write('compute vocab: %.2f seconds\n' % (time.time() - t))  # ~ 20 s
        
        t = time.time()
        keypoint_labels = vocabulary.predict(descriptors)
        sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t))  # ~ 20 s

        # visualize keypoints (color indicating label)

        # viz = img.copy()
        # for (x, y), l in zip(keypoints_arr, keypoint_labels):
        #     cv2.circle(viz, (int(x), int(y)), 3, colors[l], -1)
        # display_image(viz)

        # generate labelmap

        labelmap = np.zeros(image.shape[:2], np.int)
        keypoints_arr_int = np.floor(keypoints_arr + (xmin, ymin)).astype(np.int)  # coords on original image
        labelmap[keypoints_arr_int[:,1], keypoints_arr_int[:,0]] = keypoint_labels + 1

        create_if_not_exists(output_dir + '/' + stack)
        save_hdf(labelmap, labelmap_fp)
    
    return labelmap

In [12]:
indices_map = defaultdict(list)
hists0_allLandmarks = defaultdict(list)
hists1_allLandmarks = defaultdict(list)
hists2_allLandmarks = defaultdict(list)

for stack in ['MD589', 'MD594']:

# stack = 'MD589'
    dm = DataManager(stack=stack)

    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    # Load sample locations
    
    patches_rootdir = '/home/yuncong/CSHL_data_patches/'

    table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
    indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
    grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    patch_size, stride, w, h = grid_parameters.tolist()
    half_size = patch_size/2
    ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
    sample_locations = np.c_[xs.flat, ys.flat]

    
    progress_bar = FloatProgress(min=first_detect_sec, max=last_detect_sec)
    display(progress_bar)

#     for sec in range(first_detect_sec, first_detect_sec+2):
    for sec in range(first_detect_sec, last_detect_sec+1, 5):
    # for sec in [first_detect_sec]:

        progress_bar.value = sec

        labelmap = conmpute_labelmap(stack, sec)    

        # Compute histograms (method 2), for all levels

        num_sample = 20

        for name in labels[1:]:
            if name not in indices_allLandmarks_allSections[sec].dropna().index: 
                continue

            indices = indices_allLandmarks_allSections[sec][name]
            indices_sampled = np.random.choice(indices, min(len(indices), num_sample), replace=False)

            sample_locs_roi = sample_locations[indices_sampled]

    #         fnmap = [train_patches_jpg_dir + '/%s_%d_%d_%d.jpg' % (name, sample_i, sec, indices_sampled[sample_i])
    #                  for sample_i, (x, y) in enumerate(sample_locs_roi)]
    #         fnmap = [(sec, indices_sampled[sample_i]) for sample_i, (x, y) in enumerate(sample_locs_roi)]

            # compute level-2 histograms
            l = 2

            grid_size = patch_size / 2**l

            if l == 2:
                rx = [-2, -1, 0, 1]
                ry = [-2, -1, 0, 1]
            elif l == 1:
                rx = [-1, 0]
                ry = [-1, 0]
            elif l == 0:
                rx = [-.5]
                ry = [-.5]

            rxs, rys = np.meshgrid(rx, ry)

            patch_coords_allGrid = []

            for grid_i, (rx, ry) in enumerate(np.c_[rxs.flat, rys.flat]):

                patch_xmin = sample_locs_roi[:,0] + rx * grid_size
                patch_ymin = sample_locs_roi[:,1] + ry * grid_size
                patch_xmax = sample_locs_roi[:,0] + (rx + 1) * grid_size
                patch_ymax = sample_locs_roi[:,1] + (ry + 1) * grid_size

                patch_coords_allGrid.append([patch_xmin, patch_ymin, patch_xmax, patch_ymax])


            all_coords = np.hstack(patch_coords_allGrid)
            patch_xmin = all_coords[0]
            patch_ymin = all_coords[1]
            patch_xmax = all_coords[2]
            patch_ymax = all_coords[3]

            def compute_histogram_particular_label(i):
                m = (labelmap == i).astype(np.uint8)
                mi = cv2.integral(m)
                ci = mi[patch_ymin, patch_xmin] + mi[patch_ymax, patch_xmax] - mi[patch_ymax, patch_xmin] - mi[patch_ymin, patch_xmax]
                return ci

            t = time.time()
            hists = Parallel(n_jobs=16)(delayed(compute_histogram_particular_label)(i) for i in range(1, M+1))
            sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # ~ 18 seconds

            n_grid = (2**l)**2
            hists_arr2 = np.transpose(np.reshape(hists, (M, n_grid, -1)))
            print hists_arr2.shape

            # compute level-1 histograms based on level-2 histograms

            hists_arr1 = np.transpose([hists_arr2[:, [0,1,4,5], :].sum(axis=1),
                                       hists_arr2[:, [2,3,6,7], :].sum(axis=1),
                                       hists_arr2[:, [8,9,12,13], :].sum(axis=1),
                                       hists_arr2[:, [10,11,14,15], :].sum(axis=1)], 
                                      [1,0,2])
            print hists_arr1.shape

            # compute level-0 histograms based on level-1 histograms

            hists_arr0 = hists_arr1.sum(axis=1)
            print hists_arr0.shape

            hists2_allLandmarks[name].append(hists_arr2)
            hists1_allLandmarks[name].append(hists_arr1)
            hists0_allLandmarks[name].append(hists_arr0)

            indices_map[name].append([(stack, sec, i) for i in indices_sampled])

    #         indices_map[name].append(fnmap)

#         bp.pack_ndarray_file(hists_arr0, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l0.bp' % {'stack': stack, 'sec': sec})
#         bp.pack_ndarray_file(hists_arr1, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l1.bp' % {'stack': stack, 'sec': sec})
#         bp.pack_ndarray_file(hists_arr2, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l2.bp' % {'stack': stack, 'sec': sec})

#         save_hdf(hists_arr0, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l0.hdf' % {'stack': stack, 'sec': sec})
#         save_hdf(hists_arr1, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l1.hdf' % {'stack': stack, 'sec': sec})
#         save_hdf(hists_arr2, output_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l2.hdf' % {'stack': stack, 'sec': sec})

hists0_allLandmarks.default_factory = None
hists1_allLandmarks.default_factory = None
hists2_allLandmarks.default_factory = None

indices_map.default_factory = None

(20, 16, 200)
(20, 4, 200)
(20, 200)


done in 14.975886 seconds


(20, 16, 200)
(20, 4, 200)
(20, 200)


done in 15.108951 seconds


(20, 16, 200)
(20, 4, 200)
(20, 200)


...

(16, 16, 200)
(16, 4, 200)
(16, 200)


done in 16.025295 seconds


(20, 16, 200)
(20, 4, 200)
(20, 200)


done in 66.384332 seconds


KeyboardInterrupt: 

In [11]:
indices_map

defaultdict(None,
            {'5N': [[('MD589', 150, 15577),
               ('MD589', 150, 15174),
               ('MD589', 150, 13933),
               ('MD589', 150, 15789),
               ('MD589', 150, 14754),
               ('MD589', 150, 14956),
               ('MD589', 150, 14345),
               ('MD589', 150, 14143),
               ('MD589', 150, 14962),
               ('MD589', 150, 14545),
               ('MD589', 150, 15165),
               ('MD589', 150, 15586),
               ('MD589', 150, 14752),
               ('MD589', 150, 15173),
               ('MD589', 150, 15370),
               ('MD589', 150, 15379),
               ('MD589', 150, 13521),
               ('MD589', 150, 14137),
               ('MD589', 150, 14552),
               ('MD589', 150, 15373),
               ('MD589', 150, 13520),
               ('MD589', 150, 14141),
               ('MD589', 150, 14548),
               ('MD589', 150, 15175),
               ('MD589', 150, 14763),
               ('MD589', 1

In [ ]:
hists0_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists0_allLandmarks.iteritems()}
hists1_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists1_allLandmarks.iteritems()}
hists2_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists2_allLandmarks.iteritems()}
indices_map = {name: [(i,) + l for i, l in enumerate(chain(*indices))] for name, indices in indices_map.iteritems()}

In [ ]:
for name, arr in hists0_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(stack)s_%(name)s_histograms_l0.bp' % {'stack': stack, 'name': name})

for name, arr in hists1_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(stack)s_%(name)s_histograms_l1.bp' % {'stack': stack, 'name': name})

for name, arr in hists2_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(stack)s_%(name)s_histograms_l2.bp' % {'stack': stack, 'name': name})

In [ ]:
def save_table(data, fn):
    with open(fn, 'w') as f:
        for line in data:
            f.write(' '.join([str(c) for c in line]) + '\n')
                
def load_table(fn):
    with open(fn, 'r') as f:
        lines = f.readlines()
    data = [l.split() for l in lines] 
    return data

In [ ]:
for name, ind_map in indices_map.iteritems():
    save_table(ind_map, train_patches_jpg_dir + '/indices_map_%s.txt' % name)